In [1]:
import pandas as pd
import numpy as np
import nltk

In [2]:
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> l
Packages:
  [ ] abc................. Australian Broadcasting Commission 2006
  [ ] alpino.............. Alpino Dutch Treebank
  [ ] averaged_perceptron_tagger Averaged Perceptron Tagger
  [ ] averaged_perceptron_tagger_ru Averaged Perceptron Tagger (Russian)
  [ ] basque_grammars..... Grammars for Basque
  [ ] biocreative_ppi..... BioCreAtIvE (Critical Assessment of Information
                           Extraction Systems in Biology)
  [ ] bllip_wsj_no_aux.... BLLIP Parser: WSJ Model
  [ ] book_grammars....... Grammars from NLTK Book
  [ ] brown............... Brown Corpus
  [ ] brown_tei........... Brown Corpus (TEI XML Version)
  [ ] cess_cat............ CESS-CAT Treebank
  [

True

In [3]:
pip install transformers

     |████████████████████████████████| 3.1 MB 5.1 MB/s 
     |████████████████████████████████| 895 kB 48.4 MB/s 
     |████████████████████████████████| 59 kB 6.6 MB/s 
     |████████████████████████████████| 596 kB 65.9 MB/s 
     |████████████████████████████████| 3.3 MB 49.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv
Saving train.csv to train.csv


# Import the data

In [ ]:
df_train=pd.read_csv("train.csv")
df_test=pd.read_csv("test.csv")
# df_submission=pd.read_csv("sample_submission.csv")

In [ ]:
df_train.shape

(7613, 5)

In [ ]:
df_test.shape

(3263, 4)

In [ ]:
train_texts=df_train['text']

In [ ]:
test_texts=df_test['text']

# Zero Shot Classification to label the tweets

In [ ]:
from transformers import pipeline
zsc=pipeline("zero-shot-classification", model="valhalla/distilbart-mnli-12-3",multi_labels=True,device=0)

In [ ]:
candidate_labels = ['Profane','Contentment','Distress','Offensive','Happy','Fear','Love','Sad','Amusement','Serenity']

In [ ]:
def get_labels(texts,interest_tags):
  query_labels=[]
  for text in texts:
    interstlabels=zsc(text,interest_tags)  
    # if not isinstance(interstlabels, list):
    #   interstlabels= [interstlabels]   
    d1 = {}
    for ii in range(0,10):
      d1.update({interstlabels['labels'][ii]:interstlabels['scores'][ii]})
    query_labels.append(d1)
  return query_labels
  # return interstlabels

In [ ]:
test_features=get_labels(test_texts,candidate_labels)


/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:901: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


In [ ]:
df_te=pd.DataFrame.from_dict(test_features)
df_te['Tweets']=df_test['text']
cols=df_te.columns.values.tolist()
cols=cols[-1:] + cols[:-1]
df_te=df_te[cols]


## Tweet Feature Extraction

In [ ]:
df_te

,Tweets,Sad,Profane,Distress,Offensive,Fear,Contentment,Serenity,Happy,Amusement,Love
0,Just happened a terrible car crash,0.609298,0.293662,0.061245,0.018758,0.004726,0.003394,0.002906,0.002637,0.001992,0.001383
1,"Heard about #earthquake is different cities, s...",0.233448,0.093894,0.177116,0.062449,0.138575,0.093611,0.078481,0.066631,0.034345,0.021451
2,"there is a forest fire at spot pond, geese are...",0.365335,0.108797,0.280675,0.065158,0.057290,0.032350,0.027709,0.022934,0.022126,0.017627
3,Apocalypse lighting. #Spokane #wildfires,0.134652,0.325919,0.161785,0.206424,0.073647,0.027331,0.019187,0.021125,0.018821,0.011109
4,Typhoon Soudelor kills 28 in China and Taiwan,0.231189,0.122328,0.206523,0.209605,0.063765,0.059933,0.042258,0.027616,0.021326,0.015456
...,...,...,...,...,...,...,...,...,...,...,...
3258,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,0.042493,0.132614,0.100371,0.078248,0.131800,0.171491,0.090238,0.137075,0.076906,0.038763
3259,Storm in RI worse than last hurricane. My city...,0.346423,0.144664,0.273144,0.084222,0.047501,0.027508,0.031855,0.014373,0.016870,0.013442
3260,Green Line derailment in Chicago http://t.co/U...,0.350965,0.145144,0.249118,0.064531,0.048211,0.051422,0.031533,0.028601,0.015698,0.014778
3261,MEG issues Hazardous Weather Outlook (HWO) htt...,0.104204,0.191950,0.252334,0.064614,0.133176,0.061183,0.081685,0.045064,0.035958,0.029833


In [ ]:
df_te.to_csv("Tweets_test_features.csv",index=False)

In [ ]:
tr_cols=df_tr.columns.tolist()

In [ ]:
df_tr

,Tweets,Distress,Profane,Sad,Offensive,Fear,Serenity,Contentment,Happy,Love,Amusement
0,Our Deeds are the Reason of this #earthquake M...,0.398306,0.167271,0.163491,0.099846,0.045943,0.039811,0.027710,0.020853,0.019971,0.016797
1,Forest fire near La Ronge Sask. Canada,0.236694,0.147730,0.169228,0.104678,0.050970,0.074901,0.108579,0.045500,0.026293,0.035427
2,All residents asked to 'shelter in place' are ...,0.143571,0.142936,0.084552,0.116530,0.129434,0.099735,0.107058,0.060662,0.049147,0.066376
3,"13,000 people receive #wildfires evacuation or...",0.242634,0.173641,0.139160,0.055238,0.120053,0.054491,0.096137,0.047122,0.039543,0.031983
4,Just got sent this photo from Ruby #Alaska as ...,0.193622,0.188761,0.226275,0.060541,0.058457,0.062464,0.079309,0.049945,0.030716,0.049910
...,...,...,...,...,...,...,...,...,...,...,...
7608,Two giant cranes holding a bridge collapse int...,0.112750,0.127490,0.402825,0.092380,0.140471,0.020978,0.044440,0.030304,0.008583,0.019778
7609,@aria_ahrary @TheTawniest The out of control w...,0.123574,0.176245,0.534970,0.054879,0.027400,0.016018,0.018443,0.015872,0.004920,0.027679
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,0.096584,0.171998,0.122111,0.112578,0.085940,0.084180,0.142789,0.096251,0.044688,0.042882
7611,Police investigating after an e-bike collided ...,0.188499,0.160617,0.170200,0.069439,0.059600,0.090908,0.095353,0.058725,0.034060,0.072597


## Tweets Classification Comparison

In [4]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegressionCV as lrcv
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix


In [5]:
from google.colab import files
files.upload()

Saving Tweets_test_features.csv to Tweets_test_features.csv
Saving Tweets_train_features.csv to Tweets_train_features.csv


{'Tweets_test_features.csv': b'Tweets,Sad,Profane,Distress,Offensive,Fear,Contentment,Serenity,Happy,Amusement,Love\nJust happened a terrible car crash,0.6092975735664368,0.2936621308326721,0.06124509125947952,0.018757743760943413,0.004726157058030367,0.0033936838153749704,0.0029056218918412924,0.002636760240420699,0.0019923150539398193,0.0013829789822921157\n"Heard about #earthquake is different cities, stay safe everyone.",0.23344847559928894,0.09389356523752213,0.17711639404296875,0.06244866922497749,0.1385745257139206,0.0936112180352211,0.07848110795021057,0.06663055717945099,0.034344542771577835,0.021450884640216827\n"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all",0.3653346002101898,0.10879683494567871,0.28067514300346375,0.06515826284885406,0.05729028955101967,0.03235028311610222,0.027708783745765686,0.022933829575777054,0.022125521674752235,0.017626507207751274\nApocalypse lighting. #Spokane #wildfires,0.13465197384357452,0.3259

In [7]:
df_train=pd.read_csv('Tweets_train_features.csv')
df_test=pd.read_csv('Tweets_test_features.csv')

In [8]:
x_train=df_train.copy()
x_train.drop(['Tweets','target'],axis=1,inplace=True)
y_train=df_train['target']
x_test=df_test.copy()
x_test.drop(['Tweets'],axis=1,inplace=True)
xTrain, xTest, yTrain, yTest = train_test_split(x_train, y_train, test_size = 0.2, random_state = 0)


In [9]:
clf = lrcv(cv=5, random_state=0).fit(xTrain, yTrain)
y_pred=clf.predict(xTest)
cm = confusion_matrix(yTest, y_pred) 
print ("Confusion Matrix : \n", cm)


Confusion Matrix : 
 [[701 185]
 [282 355]]


In [10]:
target_names = ['Distress','Not Distress']
print(classification_report(yTest, y_pred, target_names=target_names))

              precision    recall  f1-score   support

    Distress       0.71      0.79      0.75       886
Not Distress       0.66      0.56      0.60       637

    accuracy                           0.69      1523
   macro avg       0.69      0.67      0.68      1523
weighted avg       0.69      0.69      0.69      1523



In [11]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(xTrain, yTrain)


GaussianNB(priors=None, var_smoothing=1e-09)

In [14]:
y_pred_GNB=model.predict(xTest)
cm_1 = confusion_matrix(yTest, y_pred_GNB) 
print ("Confusion Matrix : \n", cm_1)


Confusion Matrix : 
 [[364 522]
 [100 537]]


In [15]:
target_names = ['Distress','Not Distress']
print(classification_report(yTest, y_pred_GNB, target_names=target_names))

              precision    recall  f1-score   support

    Distress       0.78      0.41      0.54       886
Not Distress       0.51      0.84      0.63       637

    accuracy                           0.59      1523
   macro avg       0.65      0.63      0.59      1523
weighted avg       0.67      0.59      0.58      1523



In [16]:
from sklearn.naive_bayes import MultinomialNB
clf_MNB = MultinomialNB()
clf_MNB.fit(xTrain, yTrain)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [23]:
y_pred_MNB=clf_MNB.predict(xTest)
cm_2 = confusion_matrix(yTest, y_pred_MNB) 
print ("Confusion Matrix : \n", cm_2)
target_names = ['Distress','Not Distress']
print(classification_report(yTest, y_pred_MNB, target_names=target_names))

Confusion Matrix : 
 [[875  11]
 [625  12]]
              precision    recall  f1-score   support

    Distress       0.58      0.99      0.73       886
Not Distress       0.52      0.02      0.04       637

    accuracy                           0.58      1523
   macro avg       0.55      0.50      0.38      1523
weighted avg       0.56      0.58      0.44      1523



In [20]:
from sklearn.svm import SVC
clf_svc=SVC(gamma='auto')
clf_svc.fit(xTrain, yTrain)
y_pred_svc=clf_svc.predict(xTest)


In [21]:
cm_3 = confusion_matrix(yTest, y_pred_svc) 
print ("Confusion Matrix : \n", cm_3)
target_names = ['Distress','Not Distress']
print(classification_report(yTest, y_pred_svc, target_names=target_names))

Confusion Matrix : 
 [[751 135]
 [363 274]]
              precision    recall  f1-score   support

    Distress       0.67      0.85      0.75       886
Not Distress       0.67      0.43      0.52       637

    accuracy                           0.67      1523
   macro avg       0.67      0.64      0.64      1523
weighted avg       0.67      0.67      0.66      1523

